# ☯ Coding Dojo ☯

<img style="float:left" src="imgs/vamos_coletar.png">
## Regras

Ponto de participação, poderá perdido se:

    - Aluno atrasado (ver política de atraso na especificação)
    - Recusar participação como piloto/copiloto
        - Ou não querer sair do computador, quando solicitado :-)
    - Demorar para sair quando solicitado
    - Parar de participar por:
        - Uso de outro computador
        - Uso de celular
        - Usar a internet
        - Conversa


Antes de entregar, certifique-se que tenha executado todos os comandos/códigos deste Jupyter. É obrigatório que todas as saídas tenham sido apresentadas. Perda de 3 pontos caso não tenham feito isso.

# Dependencias

Se necessário, instale o python3 e o pip3 abaixo usando `sudo apt-get install python3 python3-pip`.  Execute abaixo para instalar o [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/) e o [lxml](https://lxml.de/). Qualquer linha de comando/código como a de baixo, você pode executá-la a selecionando e pressionando `ctrl+enter` ❣️.

In [1]:
!pip3 install bs4
!pip3 install lxml

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


Caso de comando não encontrado `pip3` use `pip`. 

# Coletor

<img src="imgs/arquitetura_coletor.png">

Você deverá completar a implementação das três classes abaixo para fazer o coletor: `Domain`, `Scheduler` e `PageFetcher`. Conforme figura acima, o `Scheduler` é responsável por armazenar as filas de URLs a serem requisitadas; o PageFetcher serão *threads* responsáveis por fazer as requisições das URLs obtidas por meio do escalonador (instancia da classe `Scheduler`). A classe `Domain` armazena informações importantes sobre o servidor a serem usadas no momento do escalonamento das URLs.

## Classe `Domain`

<figure style="text-align:center">
    <img src="imgs/estrutura_coletor.png">
    <caption>Fonte: Baeza-Yates e Ribeiro-Neto, 2011 </caption>
</figure>

Antes de começar a fazer o escalonador (classe `Scheduler`) devemos implementar a classe que representa os domínios. Conforme visto na figura acima, o escalonador possuirá diversas filas, uma para cada servidor.  Em nosso caso, o servidor será um domínio - que você irá implementar na classe `Domain` do arquivo `domain.py`. 

O escalonador percorrerá cada servidor e obterá o primeiro da fila do primeiro servidor acessível. O servidor é acessível obdecendo o limite de tempo entre requisições (em segundos, usando o atributo `time_limit_between_requests`). 

Nesta atividade você irá implementar a classe `Domain`, que já possui os seguintes atributos: 

- *time_limit_between_requests*: Limite entre requisições (int)
- *nam_domain*: Nome do domínio (String)
- *time_last_access*: [Objeto Datetime](https://docs.python.org/3/library/datetime.html#datetime-objects) informando da data/hora do último acesso 

O prefixo da variável sempre irá denotar o tipo da mesma.

**Atividade 1 - classe `Domain`**: Complete a classe `Domain` implementando os métodos/atributos calculados descritos a seguir. Logo após, execute o ❣️ teste unitário ❣️ para verificar se os métodos/atributos calculados foram implementados conforme esperado.

- **accessed_now**: Método que modifica o último acesso com a data/hora atual usando um [objeto datetime](https://docs.python.org/3/library/datetime.html#datetime-objects)
- **time_since_last_access**: Atributo calculado que retorna um [objeto TimeDelta](https://docs.python.org/3/library/datetime.html#timedelta-objects) com a diferença da data atual e a data do último acesso. Veja os exemplos de uso do TimeDelta na sua documentação
- **is_accessible**: Método que verdadeiro se o domínio estiver acessível

Os métodos `count_fetched_page` e `has_finished_crawl` já estão implementados. Execute o teste unitário a seguir. Veja que o comando abaixo executa o teste unitário presente no arquivo `crawler/scheduler_test.py`. Analise-o para entender como implementamos testes unitários.

In [2]:
!python3 -m crawler.scheduler_test DomainTest.test_domain

Verificando acesso a um dominio já requisitado (após espera)
aguardando 10 segundos...
.
----------------------------------------------------------------------
Ran 1 test in 10.010s

OK


**Atividade 2 - métodos `__hash__` e `__eq__` da classe `Domain`**: A fila será implementada por um dicionario ordenado ([OrderedDict](https://docs.python.org/2/library/collections.html#collections.OrderedDict)) em que a chave será um objeto da classe domínio e o valor serão uma lista de URLs. Para que seja possível a busca do domínio, você deverá implementar os métodos `__hash__` e `__eq__` de forma similar ao seguinte exemplo: 

In [3]:
from collections import OrderedDict
class Xuxu():
    def __str__(self):
        return "xuxuuu"
    
    def __hash__(self):
        return hash("xuxu")
    def __eq__(self, x):
        return x == "xuxu"
d = OrderedDict()
d[Xuxu()] = 'A'
print("xuxu" in d)

True


No nosso caso, deve ser permitido fazer a busca no dicionário pelo nome do domínio ou pelo objeto, ou seja, após a implementação desta atividade, o seguinte código irá funcionar corretamente:

In [4]:
from crawler.domain import Domain
from collections import OrderedDict

obj_domain = Domain("oi.com",10)
dic_x = OrderedDict()
dic_x[obj_domain] = "lala"
dic_x["oi.com"] = "oi"

#abaixo, em ambos os casos, será encontrado/impresso a string "oi"
print(dic_x["oi.com"])
print(dic_x[obj_domain])

oi
oi


Lembre-se que, em nosso caso, o parametro do método `__eq__` pode ser um objeto da classe `Domain` ou uma string representando o nome do domínio. Caso faça alguma modificação no código, você deverá reiniciar o kernel para que o efeito seja visto no código acima (para reiniciar, clique em Kernel->Restart). Isso ocorre que, uma vez dado import em um módulo, esse modulo não é alterado no Jupyter caso você faça uma modificação no código.

## Classe `Scheduler`

Agora você irá implementar o escalonador. Essa classe será responsável por gerir as URLs.
Para isso, será implementado as filas por servidores conforme apresentado na figura da seção anterior.
Para isso, utilizaremos um OrderedDict em que as chaves são os servidores (objeto da classe `Domain`) e o valor será uma lista de tuplas com as URLs e sua profundidade. Com o objetivo de armazenar as URLs de forma estruturada, as mesmas **não serão** string, ao invés disso, serão objetos da classe `ParseResult` - essa classe é retornada do método [urlparse](https://docs.python.org/3/library/urllib.parse.html). Veja um exemplo abaixo.



In [5]:
from crawler.domain import Domain
from crawler.scheduler import Scheduler
from urllib.parse import urlparse

dict_filas_por_servidor = {Domain("www.globo.com",10):[
                                                (urlparse("http://www.globo.com/esporte"),1),
                                                (urlparse("http://www.globo.com/noticia_um"),2),
                                                (urlparse("http://www.globo.com/noticia_dois"),3),
                                                (urlparse("http://www.globo.com/noticia_tres"),3),
                                                ],
                            Domain("www.cnpq.br",10):[(urlparse("http://www.cnpq.br/pesquisadores"),1),
                                               (urlparse("http://www.cnpq.br/bolsas"),1)]
                          }

Assim, a classe `Scheduler`, no arquivo `scheduler.py` possui os seguintes atributos: 

- `str_usr_agent`: Nome do `User agent`. Usualmente, é o nome do navegador, no nosso caso, será o nome do coletor (usualmente, terminado em `bot`)
- `page_limit`: Número de páginas a serem coletadas
- `depth_limit`: Profundidade máxima a ser coletada
- `page_count`: Quantidade de páginas já coletadas
- `dic_url_per_domain`: Fila de URLs por domínio (explicado anteriormente)
- `set_discovered_urls`: Conjunto de URLs descobertas, ou seja, que foi extraída em algum HTML e já adicionadas na fila - mesmo se já ela foi retirada da fila. A URL armazenada deve ser uma string.
- `dic_robots_per_domain`: Dicionário armazenando, para cada domínio, o objeto representando as regras obtidas no `robots.txt`

**Atividad 3 - Método `can_add_page`**: Esse método irá retornar verdadeiro caso seja possível adicionar a página `obj_url` (objeto da classe ParseResult) que foi coletada na profundidade `depth`. Para que seja possível adicionar na lista, esta pagina não deve ter sido descoberta e nem com a profundidade maior que o limite. Este método será testado por teste unitário apenas após a implementação da adição e obtenção das URLs.

**Atividade 4 método `add_new_page` - Adicionar nova página**:  Este método retorna falso caso não seja possível adicionar a página (por meio do método `can_add_page`) e verdadeiro, caso contrário. Caso seja possível, deve-se adicionar a url `obj_url` na fila `dic_url_per_domain`. Lembre-se da estrutura do dicionário `dic_url_per_domain`. Não esqueça de armazenar que esta URL já foi descoberta. Este método será testado por teste unitário apenas após a implementação da obtenção das URLs. Lembre-se de como trabalhar com [dicionários](https://daniel-hasan.github.io/cefet-web-grad/classes/python2/#mais-colecoes)

**Atividade 5 método `get_next_url` - obtém a próxima URL da fila**: Este método retona uma tupla `url,profundade` da próxima URL a ser coletada por meio da fila `dic_url_per_domain` retirando este item da fila. Lembre-se que: 
    
- Ele deverá retornar a primeira URL do primeiro servidor que estiver acessível. 
- Você deve indicar que o servidor foi acessado 
- Caso a fila deste servidor esteja vazia, elimine-o do dicionário
- Caso não encontre a URL, [coloque a Thread para esperar](https://docs.python.org/2/library/time.html#time.sleep) e, logo após, procure novamente 

Logo após, você já pode executar o teste unitário para verificar se a obtenção e adição de URLs está correta. Caso haja algum problema, existem já alguns prints comentados no código, descomente-os para verificar se o fluxo está ocorrendo da forma correta. Você deverá entregar sem a exibição desses prints (caso contrário, perderá pontos).

In [4]:
!python3 -m crawler.scheduler_test SchedulerTest.test_add_remove_page

Verificação da ordem das URLs...
Resgatando a segunda página do mesmo dominio...
Tempo esperado: 20 segundos
.
----------------------------------------------------------------------
Ran 1 test in 20.019s

OK


**Atividade 6 método `can_fetch_page`**: Este método deve retornar verdadeiro caso a url (Objeto `ParseResult`), passada como parâmetro, pode ser coletada de acordo com o `Robots.txt` do domínio. Para isso, você deve usar o [`RobotFileParser`](https://docs.python.org/3/library/urllib.robotparser.html) para requisitar a URL do `robots.txt` deste domínio. Por razões de performance, você só poderá requisitar o robots.txt uma única vez por domínio. Assim, use o `dic_robots_per_domain` apropriadamente. Execute o teste unitário abaixo para verificar seu funcionamento. 

In [2]:
!python3 -m crawler.scheduler_test SchedulerTest.test_can_fetch

.
----------------------------------------------------------------------
Ran 1 test in 0.946s

OK


**Atividade 7 - Inicialização das sementes no construtor**: No construtor da classe `Scheduler` você deve adicionar as páginas sementes na fila. Tais sementes são passadas pelo parametro `arr_urls_seeds` do construtor. 

In [5]:
!python3 -m crawler.scheduler_test SchedulerTest.test_init

.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


Logo após, utize a anotação @synchronized aonde julgar necessário para transformar em ThreadSafe.

## PageFetcher

**Atividade 8 - Efetuar a requisição - método `request_url`:** Neste método você irá fazer a requisição usando a api [requests](https://requests.readthedocs.io/en/master/). Além disso, deve-se informar no cabeçalho (em `User-Agent`) o nome do coletor. O método deverá retornar o conteúdo (em binário) apenas se o conteúdo for HTML (ver no cabeçalho da resposta o tipo do conteúdo). Caso não seja, ele deverá retornar `None`. 

In [4]:
!python3 -m crawler.page_fetcher_test PageFetcherTest.test_request_url

/usr/lib/python3/dist-packages/requests/auth.py:49: DeprecationWarning: Non-string passwords will no longer be supported in Requests 3.0.0. Please convert the object you've passed in (<class 'NoneType'>) to a string or bytes object in the near future to avoid problems.
  warnings.warn(
https://www.google.com.br/ text/html; charset=ISO-8859-1
True
https://releases.jquery.com/ text/html; charset=UTF-8
True
F
FAIL: test_request_url (__main__.PageFetcherTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/home/aluno/Downloads/ri-crawler-master/crawler/page_fetcher_test.py", line 24, in test_request_url
    self.assertIsNone(strb_jquery, 'Ao ser requisitado alguma URL de um recurso que não é HTML, deve-se retornar None')
AssertionError: b'\n<!doctype html>\n<html class="no-js" lang="en-US">\n<head data-live-domain="releases.jquery.com">\n\t<meta charset="utf-8">\n\t<meta http-equiv="X-UA-Compatible" content="IE=edge">\n\n\t

**Atividade 9 extração das URLs por meio do método `discover_links`**: A partir do conteúdo (texto em binário) `bin_str_content` deve-se extrair seus os links. O conteúdo esta em binário, pois, o BeautifulSoup, internamente, verifica o encoding por meio da [tag meta](https://www.w3schools.com/tags/att_meta_charset.asp) e converte para UTF8. A URL da página coletada é um objeto ParseResult `obj_url` que está na profundidade `depth`. 

Para cada URL extraída, você deverá retornar uma tupla com a URL (objeto ParseResult) e a sua profundidade. Utilize o comando `yield` para retornar essas tuplas. A profundidade é calculada de por meio de  `obj_url` e `depth`, conforme comentado em sala de aula. 

Para a extração dos links, use o [CSS Selector do BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#css-selectors). Seletor CSS é uma expressão que define quais elementos HTML serão estilizados pelo CSS. No *Beatifulsoup* eles são usados para selecionar quais tags serão extraídas. Veja o exemplo de uso e brinque abaixo. Caso precise, veja aqui um pouco de seletores CSS: [slides de Web](https://fegemo.github.io/cefet-front-end/classes/css2/#outros-seletores), [documentação MDN](https://developer.mozilla.org/en-US/docs/Web/CSS/CSS_Selectors).

In [10]:
from bs4 import BeautifulSoup



In [11]:
!python3 -m crawler.page_fetcher_test PageFetcherTest.test_discover_links

Simulação da extração de links da página http://www.pudim.com.br na profundidade nível 2...
E
ERROR: test_discover_links (__main__.PageFetcherTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/home/aluno/Downloads/ri-crawler-master/crawler/page_fetcher_test.py", line 39, in test_discover_links
    discovered_links = list(self.fetcher.discover_links(obj_url,2,bin_str_content))
  File "/home/aluno/Downloads/ri-crawler-master/crawler/page_fetcher.py", line 29, in discover_links
    for link in soup.select(None):
  File "/usr/lib/python3/dist-packages/bs4/element.py", line 1949, in select
    results = soupsieve.select(selector, self, namespaces, limit, **kwargs)
  File "/usr/lib/python3/dist-packages/soupsieve/__init__.py", line 144, in select
    return compile(select, namespaces, flags, **kwargs).select(tag, limit)
  File "/usr/lib/python3/dist-packages/soupsieve/__init__.py", line 67, in compile
    return cp._cached

**Atividade 10 - método que solicita uma nova URL**: Por meio do método `crawl_new_url` você deverá utilizar os métodos previamente implementados para:

- Solicitar ao escalonador uma nova URL
- Fazer a requisição e obter o resultado (em binário)
- Caso a URL seja um HTML válido, imprima esta URL e extraia os seus links

**Atividade 11 - método run**: Este método deve coletar páginas enquanto a coleta não foi finalizada

**Atividade 12 - um pequeno teste para finalizar:** Use as sementes do seu grupo e crie abaixo um escalonador e 5 PageFetchers para extrair 30 páginas. Imprima também o tempo gasto total.

# Bibliografia

Baeza-Yates, Ricardo; Ribeiro-Neto, Berthier. **Modern information retrieval: the concepts and technology behind search**. ACM Press, 2011.

Batista, Natércia ; Brandão, Michele ; Pinheiro, Michele ; Dalip, Daniel ; Moro, Mirella . **[Dados de Múltiplas Fontes da Web: coleta, integração e pré-processamento](https://sol.sbc.org.br/livros/index.php/sbc/catalog/download/8/19/58-1?inline=1)**. Minicursos do XXIV Simpósio Brasileiro de Sistemas Multimídia e Web. 1ed.: Sociedade Brasileira de Computação, 2018, v. , p. 153-192.